In [187]:
import os
import sys
import pandas as pd
import utils as utils
import numpy as np
import datetime
import datetime as dt
from pathlib import Path
pd.set_option('display.max_columns', None)


# Simplifying Tables

Takes all values from a specific date onwards. Values on all tables identified as of Booking date in Appointments table accordingly.

In [207]:
date_cut = dt.datetime(2025, 1, 1)  # CHANGE AS DESIRED

# -------------------------------------------------
SRC_DIR = Path("../data/gcc/")
SAVE_DIR = Path("../data/gcc_simplified") / date_cut.strftime("%d_%m_%Y")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

ORIGINAL_dfs = utils.GetSavedTables(SRC_DIR)

def save_pickle(df: pd.DataFrame, table_name: str) -> Path:
    out_path = SAVE_DIR / f"{table_name}.pkl"
    df.to_pickle(out_path)
    return out_path

def require_cols(df: pd.DataFrame, table: str, cols: list) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"{table} is missing columns: {missing}. Available: {list(df.columns)}")


# =================================================
# 1) Filter Episode by AdmissionDate
# =================================================
episode_table = "ML_DataModel.Episode"
df_episode = ORIGINAL_dfs[episode_table].copy()

require_cols(df_episode, episode_table, ["ID", "AdmissionDate"])

df_episode["AdmissionDate"] = pd.to_datetime(
    df_episode["AdmissionDate"], errors="coerce"
)

# Handle timezone alignment
if getattr(df_episode["AdmissionDate"].dt, "tz", None) is not None and date_cut.tzinfo is None:
    date_cut_cmp = pd.Timestamp(date_cut, tz=df_episode["AdmissionDate"].dt.tz)
else:
    date_cut_cmp = pd.Timestamp(date_cut)

df_episode_filtered = df_episode[
    df_episode["AdmissionDate"] >= date_cut_cmp
].copy()

save_pickle(df_episode_filtered, episode_table)

episode_ids = df_episode_filtered["ID"].dropna().unique()


# =================================================
# 2) Filter Appointment based on Episode IDs
# =================================================
appt_table = "ML_DataModel.Appointment"
df_appt = ORIGINAL_dfs[appt_table].copy()

require_cols(df_appt, appt_table, ["Episode", "Patient"])

df_appt_filtered = df_appt[
    df_appt["Episode"].isin(episode_ids)
].copy()

save_pickle(df_appt_filtered, appt_table)


# =================================================
# 3) Filter Patient based on remaining Appointments
# =================================================
patient_table = "ML_DataModel.Patient"
df_patient = ORIGINAL_dfs[patient_table].copy()

require_cols(df_patient, patient_table, ["ID"])
require_cols(df_appt_filtered, appt_table, ["Patient"])

patient_ids = df_appt_filtered["Patient"].dropna().unique()

df_patient_filtered = df_patient[
    df_patient["ID"].isin(patient_ids)
].copy()

save_pickle(df_patient_filtered, patient_table)


# =================================================
# 4) Filter other Episode-derived tables
# =================================================
for table in [
    "ML_DataModel.Order",
    "ML_DataModel.Diagnosis",
    "ML_DataModel.Observation",
    "ML_DataModel.Procedures",
]:
    df1 = ORIGINAL_dfs[table].copy()
    require_cols(df1, table, ["Episode"])

    df1_filtered = df1[df1["Episode"].isin(episode_ids)].copy()
    save_pickle(df1_filtered, table)


print(f"Saved simplified tables to: {SAVE_DIR.resolve()}")


Reading ..\data\gcc\ML_DataModel.Appointment.pkl
Reading ..\data\gcc\ML_DataModel.Diagnosis.pkl
Reading ..\data\gcc\ML_DataModel.Episode.pkl
Reading ..\data\gcc\ML_DataModel.Observation.pkl
Reading ..\data\gcc\ML_DataModel.Order.pkl
Reading ..\data\gcc\ML_DataModel.Patient.pkl
Reading ..\data\gcc\ML_DataModel.Procedures.pkl
Saved simplified tables to: C:\Users\da-jaramjo907\Documents\DS_model_dev\LoS_prediction\data\gcc_simplified\01_01_2025


# Tables Preprocessing

In [ ]:
SRC_DIR = Path("../data/gcc_simplified/01_01_2025")  # CHANGE AS DESIRED: Base of tables

In [64]:
ALL_dfs = utils.GetSavedTables(SRC_DIR) #

Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Appointment.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Diagnosis.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Episode.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Observation.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Order.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Patient.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Procedures.pkl


In [206]:
pd.to_datetime(ALL_dfs["ML_DataModel.Episode"].AdmissionDate).dt.year.value_counts()

2025    484
2024     85
2023     15
2021      3
2022      2
Name: AdmissionDate, dtype: int64

## Individual Tables Preprocessing
- Removing unnecesary fields, and date parsing

In [58]:
def standard_timestamp(df, date_col = None, time_col = None, default_time: str = "00:00:00"):
    """Function to standarize all dates+time the same across tables"""
    out = df.copy()
    if date_col:
        d = out[date_col].astype(str).str.strip()
        d = d.replace(["", "nan", "None"], np.nan)
    else:
        d = pd.Series(np.nan, index=out.index)

    if time_col:
        t = out[time_col].astype(str).str.strip()
        t = t.replace(["", "nan", "None"], np.nan)
    else:
        t = pd.Series(np.nan, index=out.index)
    t = t.fillna(default_time)
    combined = (d + " " + t).where(d.notnull(), np.nan)
    return pd.to_datetime(combined, errors="coerce")

### Episode

In [183]:
df = pd.read_pickle("../data/gcc_simplified/01_01_2025/ML_DataModel.Episode.pkl")
df.head()

,ID,AdmissionDate,AdmissionMethod,AdmissionReason,AdmissionTime,DischargeDate,DischargeTime,EILink,EpisodeConsultant,EpisodeLocation,EpisodeNo,Patient,Procedure,RefId,Type,Ward
1948656,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None
2405789,14412993,2021-05-12,None,None,08:31:17,None,None,None,Dr Sarah Saunders,RHEUM,O0019329484,682710,None,32424500,O,None
3046682,15059118,2021-08-16,None,None,19:37:05,None,None,None,Dr Laura McGregor,RHEUM,O0019892553,1511191,None,33230645,O,None
3742343,15759301,2022-04-29,None,None,14:01:43,None,None,None,Dr Laura McGregor,RHEUM,O0021458757,956783,None,35402933,O,None
5127143,17151723,2023-01-10,None,None,09:48:21,None,None,None,Dr Catriona Grigor,RHEUM,O0023088478,529550,None,37670645,O,None


In [184]:
# df = ALL_dfs["ML_DataModel.Episode"].copy()
df["EpisAdmissionDateTime"] = standard_timestamp(df, date_col="AdmissionDate", time_col="AdmissionTime")
df["EpisDischargeDateTime"] = standard_timestamp(df, date_col="DischargeDate", time_col="DischargeTime")
has_discharge = df["EpisDischargeDateTime"].notna()
df = df[has_discharge]
df["LOS"] = (df["EpisDischargeDateTime"] - df["EpisAdmissionDateTime"]).dt.total_seconds() / 3600


df.drop(columns=["AdmissionDate","AdmissionTime","DischargeDate","DischargeTime"],inplace=True)
df.head(5)

,ID,AdmissionMethod,AdmissionReason,EILink,EpisodeConsultant,EpisodeLocation,EpisodeNo,Patient,Procedure,RefId,Type,Ward,EpisAdmissionDateTime,EpisDischargeDateTime,LOS
8746480,21815184,None,None,None,Mr Andrew McMahon,GS,O0027130707,2092398,None,43364732,O,None,2025-01-21 11:10:08,2025-09-09 11:42:20,5544.536667
8746525,21815229,None,None,None,Mr Mark S Duxbury,GS,O0027130768,2092398,None,43364806,O,None,2025-01-28 13:29:13,2025-09-09 11:42:21,5374.218889
8746670,21815374,None,None,None,Bowel Screening Nurse Pre-Ass 2,BSS,O0027131068,2092479,None,43365252,O,None,2025-03-26 11:04:35,2025-03-27 12:17:04,25.208056


In [177]:
dt.datetime(2025,1,1)

datetime.datetime(2025, 1, 1, 0, 0)

In [185]:
df[df.EpisDischargeDateTime > dt.datetime(2025,1,1)]

,ID,AdmissionMethod,AdmissionReason,EILink,EpisodeConsultant,EpisodeLocation,EpisodeNo,Patient,Procedure,RefId,Type,Ward,EpisAdmissionDateTime,EpisDischargeDateTime,LOS
8746480,21815184,None,None,None,Mr Andrew McMahon,GS,O0027130707,2092398,None,43364732,O,None,2025-01-21 11:10:08,2025-09-09 11:42:20,5544.536667
8746525,21815229,None,None,None,Mr Mark S Duxbury,GS,O0027130768,2092398,None,43364806,O,None,2025-01-28 13:29:13,2025-09-09 11:42:21,5374.218889
8746670,21815374,None,None,None,Bowel Screening Nurse Pre-Ass 2,BSS,O0027131068,2092479,None,43365252,O,None,2025-03-26 11:04:35,2025-03-27 12:17:04,25.208056


In [158]:
len(df.ID.unique())

589

### Appointment

In [146]:
df = ALL_dfs["ML_DataModel.Appointment"].copy()

# ------ FIRST IN APPOINTMENT -------
# Useful Time regressor
df["ApptDateTime"] = standard_timestamp(df, date_col="ApptDate", time_col="ApptTime")
df["ApptBookedDate"] = standard_timestamp(df, date_col="BookedDate", time_col=None)
df["ApptBookingLag_hours"] = (df["ApptDateTime"] - df["ApptBookedDate"]).dt.total_seconds() / 3600
df.loc[df["ApptBookingLag_hours"] < -24, "ApptBookingLag_hours"] = -24
df.loc[df["ApptBookingLag_hours"] > 24*30, "ApptBookingLag_hours"] = 24*30
# Frequency encoding for High-Cardinality Variables (WARNING OF LEAKAGE) TODO: Try Target mean encoding with Cross Validation
for col in ["Service", "Resource"]:
    freq = df[col].value_counts(dropna=False)
    df[f"Appt{col}_freq"] = df[col].map(freq) # TODO: Consider Service+Resource encoding to capture duration of x provider giving y service
# One-Hot-Encoding for few-category variables
df = df.join(pd.get_dummies(df["ServiceGroup"], prefix="Appt"))
df = df.join(pd.get_dummies(df["Status"], prefix="Appt"))
# Drop columns that wont be used anymore
df.drop(columns=["RefId", "ApptDateTime", "ApptBookedDate", "ApptDate", "ApptTime", "BookedDate", "Service","Resource", "ServiceGroup", "Status"],inplace=True)

# ------ AGGREGATING FOR EPISODES -----
df["ApptCount"] = 1
one_hot_cols = [
    c for c in df.columns
    if c.startswith("Appt_") and c not in ["ApptService_freq", "ApptResource_freq"]
]

# numeric aggregates per episode
num_agg_dict = {
    "ApptBookingLag_hours": ["mean", "min", "max"],
    "ApptService_freq": ["mean", "max"],
    "ApptResource_freq": ["mean", "max"],
    "ApptCount": ["sum"],
}
numeric_agg = df.groupby("Episode").agg(num_agg_dict)
numeric_agg.columns = [
    "_".join([str(c) for c in col]).strip("_")  # col is a tuple like ("ApptBookingLag_hours", "mean")
    if isinstance(col, tuple) else str(col)
    for col in numeric_agg.columns
]
numeric_agg = numeric_agg.rename(columns={"ApptCount_sum": "ApptCount"})
ohe_agg = df.groupby("Episode")[one_hot_cols].sum()
episode_appt = numeric_agg.join(ohe_agg)
episode_appt = episode_appt.reset_index()

episode_appt.head(5)

,Episode,ApptBookingLag_hours_mean,ApptBookingLag_hours_min,ApptBookingLag_hours_max,ApptService_freq_mean,ApptService_freq_max,ApptResource_freq_mean,ApptResource_freq_max,ApptCount,Appt_New Service Group,Appt_New Urgent Service Group,Appt_Pneumococcal,Appt_Return Service Group,Appt_A,Appt_H,Appt_N,Appt_P,Appt_T,Appt_X
0,1314313,686.25,686.25,686.25,36.0,36,9.0,9,1,0,0,0,1,0,0,0,1,0,0
1,1326406,720.00,720.00,720.00,1.0,1,25.0,25,1,0,0,0,1,0,0,0,1,0,0
2,5634346,351.75,351.75,351.75,36.0,36,9.0,9,1,0,0,0,1,0,0,0,1,0,0
3,6812897,720.00,720.00,720.00,4.0,4,25.0,25,1,0,0,0,1,0,0,0,1,0,0
4,10016900,720.00,720.00,720.00,4.0,4,25.0,25,1,0,0,0,1,0,0,0,1,0,0


### Order

In [148]:
df = ALL_dfs["ML_DataModel.Order"].copy()


### Procedures

In [149]:
df = ALL_dfs["ML_DataModel.Procedures"].copy()
df

,ID,Code,Date,Episode,RefId,Time


### Diagnosis

In [150]:
df = ALL_dfs["ML_DataModel.Diagnosis"].copy()
df

,ID,CreateDate,CreateTime,Episode,ICDCode,RefId,UpdateDate,UpdateTime


### Observation

In [151]:
df = ALL_dfs["ML_DataModel.Observation"].copy()
df

,ID,CreateDate,CreateTime,Episode,Item,RefId,Status,Value
16753484,18916638,2024-10-02,12:26:00,20592269,HEIGHT,40328589||1,A,160
16753485,18916639,2024-10-02,12:26:00,20592269,WEIGHT,40328589||2,A,139
16753486,18916640,2024-10-02,12:26:00,20592269,BMI,40328589||3,None,54.297
16753487,18916641,2024-10-02,12:26:00,20592269,TC.ABW-Bauer,40328589||29,E,55.84
16753488,18916642,2024-10-02,12:26:00,20592269,TC.IBW-Matsuzawa,40328589||42,E,64


## Tables Merging

In [ ]:
# -------------------------------------------------


# CREATE AND SAVE (Appointment+Patient)
df_block1 = pd.DataFrame()
for tablename in ["Appointment","Patient"]:
    df_add = ALL_dfs[f"ML_DataModel.{tablename}"].copy()
    suffix = tablename.lower()
    for colname in df_add.columns:
        df_add.rename(columns={colname:f"{colname}_{suffix}"}, inplace=True)
    if df_block1.empty:
        df_block1 = df_add
        continue
    print(f"Adding {tablename}, previous size: {df_block1.shape}, unique appointments: {df_block1['ID_appointment'].nunique()}")
    df_block1 = df_block1.merge(df_add,
                left_on="Patient_appointment",right_on=f"ID_{suffix}",
                how="outer")
    print(f"new size: {df_block1.shape}, unique appointments: {df_block1['ID_appointment'].nunique()}")
df_block1.drop(columns=["Patient_appointment","RefId_appointment","RefId_patient"],inplace=True)
df_block1.to_pickle(SRC_DIR / "FULL_Appointments_tables.pkl")

# CREATE AND SAVE (Episode+Order+Procedures+Diagnosis+Observation)
df_block2 = pd.DataFrame()
for tablename in ["Episode","Order","Procedures","Diagnosis","Observation"]:
    df_add = ALL_dfs[f"ML_DataModel.{tablename}"].copy()
    suffix = tablename.lower()
    for colname in df_add.columns:
        df_add.rename(columns={colname:f"{colname}_{suffix}"}, inplace=True)
    if df_block2.empty:
        df_block2 = df_add
        continue
    print(f"Adding {tablename}, previous size: {df_block2.shape}, unique episodes: {df_block2['ID_episode'].nunique()}")
    df_block2 = df_block2.merge(df_add,
                left_on="ID_episode",right_on=f"Episode_{suffix}",
                how="outer")
    print(f"new size: {df_block2.shape}, unique episodes: {df_block2['ID_episode'].nunique()}")
df_block2.drop(columns=["Episode_order","Episode_procedures","Episode_diagnosis","Episode_observation","RefId_episode","RefId_order","RefId_procedures","RefId_diagnosis","RefId_observation"],inplace=True)
df_block2.to_pickle(SRC_DIR / "FULL_Episodes_tables.pkl")

# CREATE AND SAVE (merged_Episode+merged_Appointment)
df_full_base = pd.merge(df_block1,df_block2, left_on="Episode_appointment", right_on="ID_episode", how="right")
df_full_base.drop(columns=["Episode_appointment"])
df_full_base.to_pickle(SRC_DIR / "FULL_tables_merged.pkl")


Reading ..\data\gcc_simplified\01_01_2025\FULL_Appointments_tables.pkl
Reading ..\data\gcc_simplified\01_01_2025\FULL_Episodes_tables.pkl
Reading ..\data\gcc_simplified\01_01_2025\FULL_tables_merged.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Appointment.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Diagnosis.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Episode.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Observation.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Order.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Patient.pkl
Reading ..\data\gcc_simplified\01_01_2025\ML_DataModel.Procedures.pkl
Adding Patient, previous size: (1174, 11), unique appointments: 1174
new size: (1174, 19), unique appointments: 1174
Adding Order, previous size: (589, 16), unique episodes: 589
new size: (1872, 23), unique episodes: 589
Adding Procedures, previous size: (1872, 23), unique episodes: 589
new size: (1872, 29), unique 

## Tables Filtering and Aggregation operations

- Filtering values to include only fields (e.g. diagnosis, order, procedures) according to #hours after **admission time**
- Aggregation operations to deal with one-to-many table relationships

In [16]:
num_hours_filter = 6

In [15]:
df = pd.read_pickle(SRC_DIR / "FULL_tables_merged.pkl")
df.head(5)

,ID_appointment,ApptDate_appointment,ApptTime_appointment,BookedDate_appointment,Episode_appointment,Resource_appointment,Service_appointment,ServiceGroup_appointment,Status_appointment,ID_patient,City_patient,DateOfBirth_patient,Gender_patient,Nationality_patient,PatientNo_patient,PostCode_patient,ID_episode,AdmissionDate_episode,AdmissionMethod_episode,AdmissionReason_episode,AdmissionTime_episode,DischargeDate_episode,DischargeTime_episode,EILink_episode,EpisodeConsultant_episode,EpisodeLocation_episode,EpisodeNo_episode,Patient_episode,Procedure_episode,Type_episode,Ward_episode,ID_order,Category_order,Item_order,StartDate_order,StartTime_order,ID_procedures,Code_procedures,Date_procedures,Time_procedures,ID_diagnosis,CreateDate_diagnosis,CreateTime_diagnosis,ICDCode_diagnosis,UpdateDate_diagnosis,UpdateTime_diagnosis,ID_observation,CreateDate_observation,CreateTime_observation,Item_observation,Status_observation,Value_observation
0,30097649,2025-02-17,11:30:00,2025-01-16,13950059,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,O,None,144703096,Consultation,N DERM WACH,2021-06-28,13:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30097649,2025-02-17,11:30:00,2025-01-16,13950059,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,O,None,144703097,Consultation,N URG DERM WACH,2021-09-01,13:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30097649,2025-02-17,11:30:00,2025-01-16,13950059,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,O,None,144703098,Consultation,G R DERMATOLOGY,2022-01-10,09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30097649,2025-02-17,11:30:00,2025-01-16,13950059,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,O,None,144703099,Consultation,G R DERMATOLOGY,2022-07-11,09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30097649,2025-02-17,11:30:00,2025-01-16,13950059,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,O,None,144703100,Medical Illustration,Clinical Photo/Video – Adult,2022-07-11,10:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA

In [6]:
df.columns

Index(['ID_appointment', 'ApptDate_appointment', 'ApptTime_appointment',
       'BookedDate_appointment', 'Episode_appointment', 'Patient_appointment',
       'RefId_appointment', 'Resource_appointment', 'Service_appointment',
       'ServiceGroup_appointment', 'Status_appointment', 'ID_patient',
       'City_patient', 'DateOfBirth_patient', 'Gender_patient',
       'Nationality_patient', 'PatientNo_patient', 'PostCode_patient',
       'RefId_patient', 'ID_episode', 'AdmissionDate_episode',
       'AdmissionMethod_episode', 'AdmissionReason_episode',
       'AdmissionTime_episode', 'DischargeDate_episode',
       'DischargeTime_episode', 'EILink_episode', 'EpisodeConsultant_episode',
       'EpisodeLocation_episode', 'EpisodeNo_episode', 'Patient_episode',
       'Procedure_episode', 'RefId_episode', 'Type_episode', 'Ward_episode',
       'ID_order', 'Category_order', 'Episode_order', 'Item_order',
       'RefId_order', 'StartDate_order', 'StartTime_order', 'ID_procedures',
       'Cod

In [10]:
df[["ID_patient","Patient_appointment"]].nunique(axis=1).eq(1).value_counts()

True    7111
dtype: int64

In [5]:
df = pd.read_pickle("../data/gcc_simplified/01_01_2025/FULL_tables_merged.pkl")
df

,ID_appointment,ApptDate_appointment,ApptTime_appointment,BookedDate_appointment,Episode_appointment,Patient_appointment,RefId_appointment,Resource_appointment,Service_appointment,ServiceGroup_appointment,Status_appointment,ID_patient,City_patient,DateOfBirth_patient,Gender_patient,Nationality_patient,PatientNo_patient,PostCode_patient,RefId_patient,ID_episode,AdmissionDate_episode,AdmissionMethod_episode,AdmissionReason_episode,AdmissionTime_episode,DischargeDate_episode,DischargeTime_episode,EILink_episode,EpisodeConsultant_episode,EpisodeLocation_episode,EpisodeNo_episode,Patient_episode,Procedure_episode,RefId_episode,Type_episode,Ward_episode,ID_order,Category_order,Episode_order,Item_order,RefId_order,StartDate_order,StartTime_order,ID_procedures,Code_procedures,Date_procedures,Episode_procedures,RefId_procedures,Time_procedures,ID_diagnosis,CreateDate_diagnosis,CreateTime_diagnosis,Episode_diagnosis,ICDCode_diagnosis,RefId_diagnosis,UpdateDate_diagnosis,UpdateTime_diagnosis,ID_observation,CreateDate_observation,CreateTime_observation,Episode_observation,Item_observation,RefId_observation,Status_observation,Value_observation
0,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703096,Consultation,13950059,N DERM WACH,14044614||1,2021-06-28,13:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703097,Consultation,13950059,N URG DERM WACH,14044614||2,2021-09-01,13:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703098,Consultation,13950059,G R DERMATOLOGY,14044614||3,2022-01-10,09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703099,Consultation,13950059,G R DERMATOLOGY,14044614||4,2022-07-11,09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703100,Medical Illustration,13950059,Clinical Photo/Video – Adult,14044614||5,2022-07-11,10:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30097649,2025-02-17,11:30:00,2025-01-16,13950059,246125,13442||77431||1,Dr Areti Makrygeorgou,G R DERMATOLOGY,Return Service Group,P,246125,GLASGO,1933-10-06,M,None,0610336177,G61 1,1623353,13950059,2021-05-19,None,None,13:51:13,None,None,None,Dr Areti Makrygeorgou,DERM,O0019378855,246125,None,32492847,O,None,144703101,PATH: Histopathology1,13950059,Histopathology A (1 - 6 specimens),14044614||6,2022-07-11,10:27:00,NaN,NaN,